In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
hot_dog_train_dir = '../data/food11/train/hot_dog'
hot_dog_test_dir = '../data/food11/test/hot_dog'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(hot_dog_train_dir, fname) for fname in os.listdir(hot_dog_train_dir)]
image_paths += [os.path.join(hot_dog_test_dir, fname) for fname in os.listdir(hot_dog_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 838 images


In [2]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 7

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/hot_dog\1000288.jpg assigned to cluster 5
../data/food11/train/hot_dog\1011661.jpg assigned to cluster 1
../data/food11/train/hot_dog\1013916.jpg assigned to cluster 2
../data/food11/train/hot_dog\1017226.jpg assigned to cluster 0
../data/food11/train/hot_dog\1018245.jpg assigned to cluster 1
../data/food11/train/hot_dog\1023510.jpg assigned to cluster 3
../data/food11/train/hot_dog\1040579.jpg assigned to cluster 0
../data/food11/train/hot_dog\1046526.jpg assigned to cluster 3
../data/food11/train/hot_dog\1051628.jpg assigned to cluster 6
../data/food11/train/hot_dog\1051643.jpg assigned to cluster 6
../data/food11/train/hot_dog\1051711.jpg assigned to cluster 3
../data/food11/train/hot_dog\1053879.jpg assigned to cluster 6
../data/food11/train/hot_dog\1055689.jpg assigned to cluster 5
../data/food11/train/hot_dog\1057517.jpg assigned to cluster 3
../data/food11/train/hot_dog\1057882.jpg assigned to cluster 4
../data/food11/train/hot_dog\1059903.jpg assigned to cl

In [3]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(hot_dog_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(hot_dog_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(hot_dog_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(hot_dog_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Fine-grained apple_pie classification
hot_dog_train_gen = ImageDataGenerator(rescale=1./255)
hot_dog_test_gen = ImageDataGenerator(rescale=1./255)

hot_dog_train_generator = hot_dog_train_gen.flow_from_directory(
    hot_dog_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

hot_dog_test_generator = hot_dog_test_gen.flow_from_directory(
    hot_dog_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_hot_dog = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_hot_dog.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_hot_dog = base_model_hot_dog.output
x_hot_dog = GlobalAveragePooling2D()(x_hot_dog)
x_hot_dog = Dense(512, activation='relu')(x_hot_dog)
hot_dog_predictions = Dense(num_clusters, activation='softmax')(x_hot_dog)

# Create and compile the model
hot_dog_model = Model(inputs=base_model_hot_dog.input, outputs=hot_dog_predictions)
hot_dog_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
hot_dog_model.fit(
    hot_dog_train_generator,
    steps_per_epoch=hot_dog_train_generator.samples // 32,
    validation_data=hot_dog_test_generator,
    validation_steps=hot_dog_test_generator.samples // 32,
    epochs=30
)

# Save the model
hot_dog_model.save('fine_grained_hot_dog_classifier.h5')

Found 755 images belonging to 7 classes.
Found 83 images belonging to 7 classes.
Epoch 1/30
23/23 [==============================] - 7s 284ms/step - loss: 1.9820 - accuracy: 0.2628 - val_loss: 1.6886 - val_accuracy: 0.3281
Epoch 2/30
23/23 [==============================] - 6s 252ms/step - loss: 1.1912 - accuracy: 0.6238 - val_loss: 1.3631 - val_accuracy: 0.4219
Epoch 3/30
23/23 [==============================] - 5s 239ms/step - loss: 0.8265 - accuracy: 0.7925 - val_loss: 1.0245 - val_accuracy: 0.6094
Epoch 4/30
23/23 [==============================] - 6s 246ms/step - loss: 0.6110 - accuracy: 0.8714 - val_loss: 0.9030 - val_accuracy: 0.6719
Epoch 5/30
23/23 [==============================] - 6s 248ms/step - loss: 0.4743 - accuracy: 0.9156 - val_loss: 0.8880 - val_accuracy: 0.6406
Epoch 6/30
23/23 [==============================] - 6s 247ms/step - loss: 0.3738 - accuracy: 0.9544 - val_loss: 0.7284 - val_accuracy: 0.7344
Epoch 7/30
23/23 [==============================] - 6s 247ms/step -